In [1]:
import pandas as pd
import json

csvPath="../data/job profiles/2025-02-07_profiles.csv"
df=pd.read_csv(csvPath)

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('..'))
from notebooks.utils import get_job_profile_documents_per_section
from typing import List
from langchain_core.documents import Document

documents=get_job_profile_documents_per_section(csvPath)

In [ ]:
# PROFILE PER DOCUMENT

from notebooks.utils import get_job_profile_documents
documents=get_job_profile_documents(csvPath)

In [ ]:
# def dump_documents_to_console(documents, limit=10):
#     """Prints a preview of the generated documents to the console."""
#     print(f"Total documents generated: {len(documents)}\n")
#     for i, doc in enumerate(documents[:limit]):  # Limit the number of documents printed
#         print(f"Document {i + 1}:")
#         print("-" * 50)
#         print(f"Content:\n{doc.page_content}\n")
#         print(f"Metadata:\n{json.dumps(doc.metadata, indent=4)}")
#         print("=" * 50)

# # Dump documents to console
# dump_documents_to_console(documents, limit=100)  # Adjust limit as needed


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from transformers import AutoTokenizer

import chromadb

client = chromadb.PersistentClient("../job_profiles_db2")
collection = client.get_or_create_collection("job_profiles",metadata={"hnsw:batch_size":10000})

def create_vectorstore_with_batching(documents, batch_size=100):  # Reduced batch size
    embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-small")
    
    vectorstore = Chroma(
        # persist_directory="job_profiles_db",
        client=client,
        embedding_function=embeddings,
        collection_name="job_profiles"
    )
    
    try:
        for i in range(0, len(documents), batch_size):
            batch = documents[i:i + batch_size]
            print(f"Processing batch {i//batch_size + 1}, size: {len(batch)}")
            
            vectorstore.add_documents(documents=batch)
            
            # Add verification step
            current_count = vectorstore._collection.count()
            print(f"Current document count: {current_count}")
            
    except Exception as e:
        print(f"Error during processing: {str(e)}")
        
    return vectorstore

# loader = CSVLoader(file_path="../data/job profiles/2025-02-07_profiles.csv", content_columns=["title", "overview"], encoding="utf-8-sig")
# documents = loader.load()


# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=200,
#     separators=["\n\n", "\n", " ", ""]
# )

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    AutoTokenizer.from_pretrained("thenlper/gte-small"),
    chunk_size=512,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True
)

chunks = text_splitter.split_documents(documents)

# chunks = documents
print(f"Documents adding: {len(chunks)}")
print('creating vector store..')
vectorstore = create_vectorstore_with_batching(chunks)

print(f"Collection count: {vectorstore._collection.count()}")

Documents adding: 2136
creating vector store..
Processing batch 1, size: 100
Current document count: 100
Processing batch 2, size: 100
Current document count: 200
Processing batch 3, size: 100
Current document count: 300
Processing batch 4, size: 100
Current document count: 400
Processing batch 5, size: 100
Current document count: 500
Processing batch 6, size: 100
Current document count: 600
Processing batch 7, size: 100
Current document count: 700
Processing batch 8, size: 100
Current document count: 800
Processing batch 9, size: 100
Current document count: 900
Processing batch 10, size: 100
Current document count: 1000
Processing batch 11, size: 100
Current document count: 1100
Processing batch 12, size: 100
Current document count: 1200
Processing batch 13, size: 100
Current document count: 1300
Processing batch 14, size: 100
Current document count: 1400
Processing batch 15, size: 100
Current document count: 1500
Processing batch 16, size: 100
Current document count: 1600
Processing 